In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Jupyter").getOrCreate()

spark

In [ ]:
df = spark.read.csv("/home/iceberg/warehouse/Customers.csv", header=True, inferSchema=True)


In [ ]:
df.show(5)

In [ ]:
df.write.saveAsTable("default.source_customers", mode="overwrite")

In [ ]:
df.write.saveAsTable("iceberg.target_customers")

In [ ]:
%%sql
with cdc as(     
SELECT COALESCE(s.CustomerID, t.CustomerID) AS CustomerID,
       s.Gender AS Gender,
       s.Age AS Age,
       s.`Annual Income ($)` AS `Annual Income ($)`,
       s.`Spending Score (1-100)` AS `Spending Score (1-100)`,
       s.Profession AS Profession,
       s.`Work Experience` AS `Work Experience`,
       s.`Family Size` AS `Family Size`,
       CASE
            WHEN s.CustomerID IS NULL THEN 'D'
            WHEN t.CustomerID IS NULL THEN 'I'
            ELSE 'U' 
       END AS CDC_flag
FROM 
    nyc.source_customers AS s 
FULL OUTER JOIN 
    nyc.target_customers AS t
ON 
    s.CustomerID = t.CustomerID
WHERE( s.CustomerID is null or
    
       t.CustomerID is null or    
    
    NOT (
           s.Gender = t.Gender AND
           s.Age = t.Age AND
           s.`Annual Income ($)` = t.`Annual Income ($)` AND
           s.`Spending Score (1-100)` = t.`Spending Score (1-100)` AND
           s.Profession = t.Profession AND
           s.`Work Experience` = t.`Work Experience` AND
           s.`Family Size` = t.`Family Size`
          )    
    )
    )
select * from cdc

In [ ]:
%%sql
with cdc as(     
SELECT COALESCE(s.CustomerID, t.CustomerID) AS CustomerID,
       s.Gender AS Gender,
       s.Age AS Age,
       s.`Annual Income ($)` AS `Annual Income ($)`,
       s.`Spending Score (1-100)` AS `Spending Score (1-100)`,
       s.Profession AS Profession,
       s.`Work Experience` AS `Work Experience`,
       s.`Family Size` AS `Family Size`,
       CASE
            WHEN s.CustomerID IS NULL THEN 'D'
            WHEN t.CustomerID IS NULL THEN 'I'
            ELSE 'U' 
       END AS CDC_flag
FROM 
    nyc.source_customers AS s 
FULL OUTER JOIN 
    nyc.target_customers AS t
ON 
    s.CustomerID = t.CustomerID
WHERE( s.CustomerID is null or
    
       t.CustomerID is null or    
    
    NOT (
           s.Gender = t.Gender AND
           s.Age = t.Age AND
           s.`Annual Income ($)` = t.`Annual Income ($)` AND
           s.`Spending Score (1-100)` = t.`Spending Score (1-100)` AND
           s.Profession = t.Profession AND
           s.`Work Experience` = t.`Work Experience` AND
           s.`Family Size` = t.`Family Size`
          )    
    )
    )
    
MERGE into nyc.target_customers 
using cdc 
on nyc.target_customers.CustomerID = cdc.CustomerID 
WHEN MATCHED AND cdc.CDC_flag = 'D' THEN DELETE
WHEN MATCHED AND cdc.CDC_flag = 'U' THEN UPDATE SET *
    -- iceberg_taxis.vendorID = cdc.SvendorID
WHEN NOT MATCHED THEN INSERT *
--where s.vendorID = 3

In [ ]:
%%sql
select * from nyc.target_customers

In [ ]:
%%sql
select count(*) 
from 
    nyc.source_customers

In [ ]:
%%sql
select count(*) 
from 
    nyc.target_customers